Unfortunately, both OLS & KNN regression performed poorly. I attempted to normalize the data in order to see if it changed the results. This had no effect on OLS but it did have a significant positive effect on the KNN model (though accuracy was still low: .34 for both weighted and unweighted).

In [6]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [17]:
df = pd.read_csv("movies.csv", encoding='latin1')

In [42]:
normalized_df = (df[['budget', 'gross', 'runtime', 'votes', 'year','score']]-df[['budget', 'gross', 'runtime', 'votes', 'year','score']].mean())/df[['budget', 'gross', 'runtime', 'votes', 'year','score']].std()

In [43]:
df.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [44]:
df.describe()

,budget,gross,runtime,score,votes,year
count,6.820000e+03,6.820000e+03,6820.00000,6820.000000,6.820000e+03,6820.000000
mean,2.458113e+07,3.349783e+07,106.55132,6.374897,7.121952e+04,2001.000293
std,3.702254e+07,5.819760e+07,18.02818,1.003142,1.305176e+05,8.944501
min,0.000000e+00,7.000000e+01,50.00000,1.500000,2.700000e+01,1986.000000
25%,0.000000e+00,1.515839e+06,95.00000,5.800000,7.665250e+03,1993.000000
50%,1.100000e+07,1.213568e+07,102.00000,6.400000,2.589250e+04,2001.000000
75%,3.200000e+07,4.006534e+07,115.00000,7.100000,7.581225e+04,2009.000000
max,3.000000e+08,9.366622e+08,366.00000,9.300000,1.861666e+06,2016.000000


In [45]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [52]:
normalized_df['score'] = normalized_df['score'].values.reshape(-1,1)
X = normalized_df[['budget', 'gross', 'runtime', 'votes', 'year']]
y = normalized_df['score']
regr = linear_model.LinearRegression() 
regr.fit(X,y)
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, y))


Coefficients: 
 [-0.27293812  0.00183981  0.33253186  0.41037082  0.06110886]

Intercept: 
 1.8416468746109056e-14

R-squared:
0.28822445887934145


In [53]:
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(n_neighbors=10)
knn_w = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
X = normalized_df[['budget', 'gross', 'runtime', 'votes', 'year']]
Y = normalized_df['score']
knn.fit(X, Y)

from sklearn.model_selection import cross_val_score

score = cross_val_score(knn, X, Y, cv=5)
print("Unweighted Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

score_w = cross_val_score(knn_w, X, Y, cv=5)
print("Weighted Accuracy: %0.2f (+/- %0.2f)" % (score_w.mean(), score_w.std() * 2))

Unweighted Accuracy: 0.34 (+/- 0.16)
Weighted Accuracy: 0.34 (+/- 0.17)
